### CPSC 672 Network Project

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from random import sample
import matplotlib.pyplot as plt
import matplotlib as mpl
import ast

%matplotlib inline